In [1]:
import numpy as np
from ortools.linear_solver import pywraplp
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

from itertools import product
import pandas as pd
import math

from vsim.utils import get_processed_metadata

In [2]:
# Load data from the Excel file
file_path = "../data/VOSimu-InputInformation.xlsx"

loc_df, vehicles_df, co_df = get_processed_metadata(meta_file_path=file_path)

In [22]:
# Helper function to calculate Manhattan distance
def manhattan_distance(x1, y1, x2, y2):
    return abs(x1 - x2) + abs(y1 - y2)

In [25]:
class VSDataCenter:
    def __init__(self, data_file):
        self._locations = {}
        self._vehicles = {}
        self._container_orders = {}
        self._distance_matrix = None

        self._read_data(data_file)
        self._create_distance_matrix()

    def _read_data(self, data_file):
        locations_df, vehicles_df, container_orders_df = get_processed_metadata(meta_file_path=data_file)
        self._locations = locations_df.set_index("location_name").to_dict("index")
        self._vehicles = vehicles_df.set_index("id").to_dict("index")
        self._container_orders = container_orders_df.set_index("co_id").to_dict("index")

    def _create_distance_matrix(self):
        num_locations = len(self._locations)
        distance_matrix = np.zeros((num_locations, num_locations))
        locations = list(self._locations.values())
        for i, j in product(range(num_locations), range(num_locations)):
            x1, y1, x2, y2 = locations[i]['x'], locations[i]['y'], locations[j]['x'], locations[j]['y']
            distance_matrix[i, j] = manhattan_distance(x1, y1, x2, y2)

        self._distance_matrix = distance_matrix

    def get_distance(self, loc_1, loc_2):
        loc_1_idx = self.locations.keys().index(loc_1)
        loc_2_idx = self.locations.keys().index(loc_2)
        return self.distance_matrix[loc_1_idx][loc_2_idx]

    @property
    def locations(self): return self._locations.copy()

    @property
    def vehicles(self): return self._vehicles.copy()

    @property
    def container_orders(self): return self._container_orders.copy()

    @property
    def distance_matrix(self): return np.copy(self._distance_matrix)




In [32]:
data_center = VSDataCenter(data_file=file_path)

In [ ]:
class VSSolver:
    def __init__(self, data_center):
        self._data_center: VSDataCenter = data_center

        self._solver = None
        self._var_x = None
        self._var_y = None

        self._opt_obj = None
        self._opt_x = None

    def build_model(self):
        self._solver = pywraplp.Solver.CreateSolver('SCIP')

        self._create_variables()
        self._create_objective()
        self._create_constraints()

    def optimize(self):
        self._solver.SetTimeLimit(20000)
        status = self._solver.Solve()
        if status == pywraplp.Solver.OPTIMAL:
            vehicles = self._data_center.vehicles
            orders = self._data_center.container_orders
            self._opt_obj = self._solver.Objective().Value()
            self._opt_x = {(v, o): 0 for v in vehicles for o in orders}
            for v, o in product(vehicles, orders):
                self._opt_x[v, o] = self._var_x[v, o].solution_value()

    def _create_variables(self):
        vehicles = self._data_center.vehicles
        orders = self._data_center.container_orders

        self._var_x = {}
        for v in vehicles:
            for o in orders:
                self._var_x[v, o] = self._solver.IntVar(0, 1, f'x[{v},{o}]')

    def _create_objective(self):
        vehicles = self._data_center.vehicles
        orders = self._data_center.container_orders

        obj_expr = []
        for v in vehicles:
            for o in orders:
                v_loc = v['start_location']
                o_origin = o['origin']
                o_dest = o['dest']

                v_to_origin = self._data_center.get_distance(v_loc, o_origin)
                origin_to_dest = self._data_center.get_distance(o_origin, o_dest)

                obj_expr.append((v_to_origin + origin_to_dest) * self._var_x[v, o])

        self._solver.Minimize(self._solver.Sum(obj_expr))

    def _create_constraints(self):
        locations = self._data_center.locations
        vehicles = self._data_center.vehicles
        orders = self._data_center.container_orders

        # C1: Each order must be assigned to exactly one vehicle        
        for o in orders:
            self._solver.Add(
                self._solver.Sum(self._var_x[v, o] for v in vehicles) == 1
            )

        # C2: Number of vehicles dispatched to a location should not exceed its capacity
        for loc, loc_data in locations.items():
            expr = [
                self._var_x[v, o]
                for v in vehicles
                for o in orders
                if o['origin'] == loc
            ]
            self._solver.Add(
                self._solver.Sum(expr) <= loc_data['capacity']
            )